In [2]:
import sqlite3
import csv
import pandas as pd
import os
import re

import math
from tqdm import tqdm
import multiprocessing as mp
import time
import sched
import ast

folder_prefix = '/home/jovyan/work/'


In [24]:
full_250_df = pd.read_pickle(folder_prefix + "charters_full_2015_15_250.pkl")


In [25]:
full_250_df

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,SEARCH16,TOTETH,PCTETH,AGE,PCTFRL,PLACE,STR,WEBTEXT_METHOD,REPLACED,CMO_REPLACED
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,3,False,False
1,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,3,False,False
2,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,3,False,False
3,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,3,False,False
4,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,3,False,False
5,Arizona Agribusiness & Equine Center,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,3,False,False
6,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY MIDDLE,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,"CUMBERLAND ACADEMY MIDDLE 1040 SHILOH RD , TY...",240.0,0.471513,2.0,0.379175,City,12.469378,3,False,False
7,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND H S,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,"CUMBERLAND H S 7200 PALUXY RD , TYLER TX 7570...",59.0,0.385621,0.0,NaN,Rural,NaN,3,False,False
8,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,"CUMBERLAND ACADEMY 1340 SHILOH RD , TYLER TX ...",217.0,0.434870,17.0,0.408818,City,13.773116,3,False,False
9,ACCELERATED INTERMEDIATE ACADEMY,281.0,ACCELERATED INTERDISCIPLINARY ACAD,TX,3.0,http://www.aiacharterschools.org/,1.0,TX,NaN,NaN,...,ACCELERATED INTERDISCIPLINARY ACAD 12825 SUMM...,276.0,0.982206,11.0,0.864769,City,16.676558,3,False,False


In [3]:
parsed_df_5 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_5.csv", sep="\t", low_memory=False, encoding="utf-8")

In [4]:
parsed_df_4 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_4.csv", sep="\t", low_memory=False, encoding="utf-8")

In [5]:
parsed_df_3 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_3.csv", sep="\t", low_memory=False, encoding="utf-8")

In [6]:
parsed_df_2 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_2.csv", sep="\t", low_memory=False, encoding="utf-8")

In [7]:
parsed_df_1 = pd.read_csv(folder_prefix + 'nowdata/parsing/parsed_df_1.tar.gz', compression='gzip', sep='\t', low_memory=False, encoding="utf-8")


In [79]:
lookup = pd.DataFrame(columns=['NCESSCH','OVERLAPS_REMOVED','DATA_SOURCE'])

In [11]:
for i in parsed_df_1.columns:
    
    print(i)
    



parsed_df_1.csv
CMO_MEMSUM
SCH_NAME
CMO_STATE
CMO_SCHNUM
CMO_URL
CMO_NUMSTATES
CMO_ALLSTATES
CMO_SECTOR
CMO_NUMSTUDENTS_CREDO17
CMO_TYPE
CMO_WEBTEXT
SURVYEAR
FIPST
STABR
SEANAME
LEAID
ST_LEAID
SCHID
ST_SCHID
NCESSCH
MSTREET1
MSTREET2
MSTREET3
MCITY
MSTATE
MZIP
MZIP4
PHONE
LSTREET1
LSTREET2
LSTREET3
LCITY
LSTATE
LZIP
LZIP4
UNION
OUT_OF_STATE_FLAG
SCH_TYPE_TEXT
SCH_TYPE
RECON_STATUS
GSLO
GSHI
LEVEL
VIRTUAL
BIES
SY_STATUS_TEXT
SY_STATUS
UPDATED_STATUS_TEXT
UPDATED_STATUS
EFFECTIVE_DATE
CHARTER_TEXT
G13OFFERED
AEOFFERED
UGOFFERED
NOGRADES
CHARTAUTH1
CHARTAUTHN1
CHARTAUTH2
CHARTAUTHN2
IGOFFERED
WEBSITE
FRELCH
REDLCH
AE
TOTAL
AM
AMALM
AMALF
AS
ASALM
ASALF
HI
HIALM
HIALF
BL
BLALM
BLALF
WH
WHALM
WHALF
HP
HPALM
HPALF
TR
TRALM
TRALF
TITLEI_TEXT
TITLEI_STATUS
STITLEI
SHARED_TIME
MAGNET_TEXT
NSLPSTATUS_TEXT
NSLPSTATUS_CODE
NAME
OPSTFIPS
LSTREE
STFIP15
CNTY15
NMCNTY15
LOCALE15
LAT1516
LON1516
CBSA15
NMCBSA15
CBSATYPE15
CSA15
NMCSA15
NECTA15
NMNECTA15
CD15
SLDL15
SLDU15
LSTREET114
LSTREET214
LSTREET

In [17]:
so_far = parsed_df_1['NCESSCH'].append(parsed_df_2['NCESSCH']).append(parsed_df_3['NCESSCH']).append(parsed_df_4['NCESSCH']).append(parsed_df_5['NCESSCH'])

In [56]:
so_far_unique = list(set(so_far))

In [58]:
len(so_far_unique)

6037

In [77]:
len(so_far)

6475

In [60]:
cleaned_list = [x for x in so_far_unique if str(x) != 'nan']


In [62]:
len(cleaned_list) # true unique ids that have been seen already

6036

In [76]:
parsed_df_1.shape[0]+parsed_df_2.shape[0]+parsed_df_3.shape[0]+parsed_df_4.shape[0]+parsed_df_5.shape[0]

6475

In [80]:
lookup['NCESSCH'] = full_250_df['NCESSCH']

In [90]:
count = 0
minus = 0
u = set(full_250_df['NCESSCH'])
for nc in u:
    indices = lookup.index[lookup['NCESSCH'] == nc].tolist()
    if nc in cleaned_list:
        for i in indices:
            lookup.loc[i, 'OVERLAPS_REMOVED'] = 1
            lookup.loc[i, 'DATA_SOURCE'] = 'full'
        count+=1
    else:
        
        for i in indices:
            lookup.loc[i, 'OVERLAPS_REMOVED'] = 0
            lookup.loc[i, 'DATA_SOURCE'] = 'full_15_250'
        minus+=1
        
        
        

In [91]:
count

6036

In [92]:
minus

4929

In [93]:
count+minus

10965

In [67]:
lookup

,NCESSCH,OVERLAPS_REMOVED,DATA_SOURCE
0,4.008280e+10,0,full_15_250
1,4.008280e+10,0,full_15_250
2,4.004170e+10,0,full_15_250
3,4.004170e+10,0,full_15_250
4,4.001060e+10,0,full_15_250
5,4.001060e+10,0,full_15_250
6,4.800034e+11,1,full
7,4.800034e+11,1,full
8,4.800034e+11,1,full
9,4.800192e+11,1,full


In [97]:
lookup.shape[0]

11521

In [98]:
lookup[lookup['OVERLAPS_REMOVED'] == 1].shape[0] #this number is bigger than len(cleaned_list) cz lookup has repeated rows
#count is 6036 which means it made 1, 6036 times -> which is the leangth of cleaned list


6242

In [85]:
len(so_far)

6475

In [96]:
lookup.isna().sum()

NCESSCH             0
OVERLAPS_REMOVED    0
DATA_SOURCE         0
dtype: int64

In [101]:
len(set(so_far))

6037

In [104]:
lookup.to_csv(folder_prefix + "lookup.csv", mode="w", index=False, header=lookup.columns.values, sep="\t", encoding="utf-8")

In [106]:
s = pd.read_csv(folder_prefix + "lookup.csv", sep="\t", low_memory=False, encoding="utf-8")

In [107]:
s

,NCESSCH,OVERLAPS_REMOVED,DATA_SOURCE
0,4.008280e+10,0,full_15_250
1,4.008280e+10,0,full_15_250
2,4.004170e+10,0,full_15_250
3,4.004170e+10,0,full_15_250
4,4.001060e+10,0,full_15_250
5,4.001060e+10,0,full_15_250
6,4.800034e+11,1,full
7,4.800034e+11,1,full
8,4.800034e+11,1,full
9,4.800192e+11,1,full


In [108]:
s[s['OVERLAPS_REMOVED'] == 1].shape[0]

6242

In [109]:
s.isna().sum()

NCESSCH             0
OVERLAPS_REMOVED    0
DATA_SOURCE         0
dtype: int64

In [111]:
unseen = s[s['OVERLAPS_REMOVED'] == 0]
unseen

,NCESSCH,OVERLAPS_REMOVED,DATA_SOURCE
0,4.008280e+10,0,full_15_250
1,4.008280e+10,0,full_15_250
2,4.004170e+10,0,full_15_250
3,4.004170e+10,0,full_15_250
4,4.001060e+10,0,full_15_250
5,4.001060e+10,0,full_15_250
27,4.200839e+11,0,full_15_250
28,4.200043e+11,0,full_15_250
29,1.200180e+11,0,full_15_250
30,1.200180e+11,0,full_15_250


In [112]:
5279 + 6242



11521

In [113]:
li = list(unseen['NCESSCH'])

In [115]:
len(li)

5279

In [116]:
p = full_250_df[full_250_df['NCESSCH'].isin(li)]

In [117]:
p

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,SEARCH16,TOTETH,PCTETH,AGE,PCTFRL,PLACE,STR,WEBTEXT_METHOD,REPLACED,CMO_REPLACED
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,3,False,False
1,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,3,False,False
2,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,3,False,False
3,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,3,False,False
4,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,3,False,False
5,Arizona Agribusiness & Equine Center,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,3,False,False
27,ASPIRA of PA,3966.0,JOHN B. STETSON CHARTER SCHOOL,PA,5.0,http://www.aspirapa.org/,1.0,PA,CMO,3494.0,...,JOHN B. STETSON CHARTER SCHOOL 3200 B Street ...,849.0,0.988359,5.0,0.912689,City,12.632353,3,False,False
28,ASPIRA of PA,3966.0,EUGENIO MARIA DE HOSTOS CS,PA,5.0,http://www.aspirapa.org/,1.0,PA,CMO,3494.0,...,EUGENIO MARIA DE HOSTOS CS 6310 N 2nd Street ...,477.0,0.993750,17.0,0.968750,City,12.972973,3,False,False
29,Academica,48541.0,SOMERSET ACADEMY KEY CHARTER HIGH SCHOOL,FL,108.0,http://www.academica.org/,6.0,"FL,UT,CA,NV,TX,DC",VOS,23788.0,...,SOMERSET ACADEMY KEY CHARTER HIGH SCHOOL 959 ...,NaN,NaN,-1.0,NaN,NaN,NaN,3,False,False
30,Somerset Academy,23411.0,SOMERSET ACADEMY KEY CHARTER HIGH SCHOOL,NY,67.0,http://www.somersetacademyschools.com/,1.0,NY,Hybrid,14951.0,...,SOMERSET ACADEMY KEY CHARTER HIGH SCHOOL 959 ...,NaN,NaN,-1.0,NaN,NaN,NaN,3,False,False
